# AI Model for Quantum Computing @ TSpark 2022 Quantum+ Camp

## Install Prerequisites

In [ ]:
!pip install --pre deepchem

## Import Libraries

In [2]:
import deepchem as dc
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

## Traning Model

In [ ]:
tasks, datasets, transformers = dc.molnet.load_tox21(featurizer='ECFP')
train_dataset, valid_dataset, test_dataset = datasets

wcol = train_dataset.w[:,1]
train_num = np.nonzero(wcol)
x_train = train_dataset.X[train_num]
y_train = train_dataset.y[train_num][:,1]
w_train = train_dataset.w[train_num][:,1]

BATCH_SIZE = 64

wcol = valid_dataset.w[:,1]
train_num = np.nonzero(wcol)
x_val = valid_dataset.X[train_num]
y_val = valid_dataset.y[train_num][:,1]
w_val = valid_dataset.w[train_num][:,1]

wcol = test_dataset.w[:,1]
train_num = np.nonzero(wcol)
x_test = test_dataset.X[train_num]
y_test = test_dataset.y[train_num][:,1]
w_test = test_dataset.w[train_num][:,1]

print(x_val.shape)
print(x_train.shape)

train_dataset1 = tf.data.Dataset.from_tensor_slices((x_train, y_train, w_train))
train_dataset1 = train_dataset1.shuffle(buffer_size=1024).batch(BATCH_SIZE)

val_dataset1 = tf.data.Dataset.from_tensor_slices((x_val, y_val, w_val))
val_dataset1 = val_dataset1.shuffle(buffer_size=1024).batch(BATCH_SIZE)

test_dataset1 = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_dataset1 = test_dataset1.shuffle(buffer_size=1024).batch(BATCH_SIZE)

inputs = keras.Input(shape=(1024,), name="digits")
x1 = keras.layers.Dense(12, activation="relu")(inputs)
x2 = keras.layers.Dropout(.2)(x1)
x3 = keras.layers.Dense(6, activation="relu")(x2)
outputs = keras.layers.Dense(1, activation="sigmoid", name="predictions")(x3)
model = keras.Model(inputs=inputs, outputs=outputs)
model.summary()
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=1e-3),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=["accuracy",tf.keras.metrics.AUC()])

model.fit(train_dataset1, epochs=50, validation_data=val_dataset1,verbose=1)

## Test

In [ ]:
testing_result = model(x_test)
ans = 0
for i in range(len(testing_result)):
  if abs(testing_result[i]-y_test[i])<=0.5 :
    ans+=1
print(ans/len(testing_result))

auc_m = tf.keras.metrics.AUC()
auc_m.update_state(y_test, testing_result)
auc_m.result().numpy()